# Theis Example

The objective of this exercise is to learn how to use Python to help calculate drawdown using the Theis equation.  We will be plotting results along the way using matplotlib.

Steps

1. Construct and test a function that allows us to calculate the Theis solution
2. Call the function in a loop and make a plot of drawdown versus distance and drawdown versus time
3. Repeat step 2, but using numpy broadcasting instead
4. Learn how to use the numpy.meshgrid function to create and plot 2d fields
5. Create a two-dimensional contour map and filled contour map of drawdown at a pumping well
6. Create an animation that shows the propagation of pumping at a single well
7. Create a composite drawdown map using a list of wells and pumping rates provided in a table

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Step 1. Construct and est a Theis function

The Theis (1935) equation is used to calculate drawdown for two-dimensional radial groundwater flow to a point source in an infinite, homogeneous aquifer. The Theis equation was derived from heat transfer literature (with the mathematical help of C.I. Lubin) and is defined as:

\begin{equation}
s = \frac{Q}{4 \pi T} W(u)
\end{equation}

where  
$s$ is drawdown [L],  
$Q$ is the pumping rate [L$^3$/T],  
$T$ is the aquifer transmissivity [L$^2$/T],  
$u$ is a dimensionless time parameter [unitless], and  
$W(u)$ is the Well function (exponential integral $E_1$) [unitless]. The exponential integral is available in ``scipy.special`` as the ``exp1()`` function.

The dimensionless time parameter is defined as:

\begin{equation}
u = \frac{r^2S}{4Tt}
\end{equation}

where  
$r$ is the distance from the pumping well to a point where drawdown is observed [L],   
$S$ is storativity [unitless], and  
$t$ is the time since pumping began. 

Storativity is defined as:

\begin{equation}
S = S_s b
\end{equation}

where  
$S_s$ is specific storage [1/L] and   
$b$ is the thickness of the aquifer.

To get started with this, we can assume the following parameters:

```
Q = 1000. cubic feet per day
T = 1000. feet squared per day
r = 1000. feet
S = 0.0001 unitless
t = 100. days
```

Hint: you should get a calculated drawdown of 0.614.

In [ ]:
from scipy.special import exp1
import numpy as np
def theis_drawdown(Q, T, r, S, t):
    u = r ** 2 * S / 4. / T / t
    wu = exp1(u)
    s = Q / 4. / np.pi / T * wu
    return s

In [ ]:
# test the function with some different values
Q = 1000. # cubic feet per day
T = 1000. # feet squared per day
r = 1000. # feet
S = 0.0001 # unitless
t = 100. # days
theis_drawdown(Q, T, r, S, t)

## Step 2. Call the function in a loop

Create plots of drawdown versus distance and drawdown versus time, by calling the Theis function multiple times in a loop.

In [ ]:
# call the function in a loop and plot the results
x = []
y = []
for r in np.arange(1, 100000, 1000):
    x.append(r)
    y.append(theis_drawdown(Q, T, r, S, t))

import matplotlib.pyplot as plt
plt.plot(x, y, "bo-")
plt.xlabel("distance, in feet")
plt.ylabel("drawdown, in feet")

In [ ]:
x = []
y = []
for t in np.arange(1, 100000, 1000):
    x.append(t)
    y.append(theis_drawdown(Q, T, r, S, t))

import matplotlib.pyplot as plt
plt.plot(x, y, "bo-")
plt.xlabel("time, in days")
plt.ylabel("drawdown, in feet")

## Step 3.  Repeat step 2, but using numpy broadcasting instead of loops

In [ ]:
# use broadcasting instead
t = np.arange(1, 100000, 1000)
s = theis_drawdown(Q, T, r, S, t)
plt.plot(t, s)

## Step 4. Learn how to use numpy.meshgrid

Use numpy.meshgrid to create an x and y grid from 0 to 10000 using a spacing of 20.  If we have a well located at an (x, y) postion of (2500, 2500), calculate the distance to the well for every point in the meshgrid.  Make a plot of the distance using `contourf`.  

In [ ]:
# what about a spatial plot of drawdown?
# start by creating x,y grid
x = np.linspace(0, 10000, 20)
y = np.linspace(0, 10000, 20)
xg, yg = np.meshgrid(x, y)
ax = plt.subplot(1, 1, 1, aspect="equal")
ax.contourf(xg, yg, yg)

In [ ]:
# for each square, assign the distance from the cell to the well
xwell = 2500.
ywell = 2500.
r = np.sqrt((xg - xwell) ** 2 + (yg - ywell) ** 2)
ax = plt.subplot(1, 1, 1, aspect="equal")
ax.contourf(xg, yg, r)

## Step 5. Create a two-dimensional plot of drawdown

Use the following parameters
* Q = 1000. cubic feet per day
* T = 1000. feet squared per day
* r = calculated as the distance to the well at (2500, 2500)
* S = 0.0001 unitless
* t = 100. days

In [ ]:
xwell = 2500.
ywell = 2500.
x = np.linspace(0, 10000, 200)
y = np.linspace(0, 10000, 200)

xg, yg = np.meshgrid(x, y)
Q = 1000. # cubic feet per day
T = 1000. # feet squared per day
# r = 1000. # feet
r = np.sqrt((xg - xwell) ** 2 + (yg - ywell) ** 2)
S = 0.0001 # unitless
t = 100. # days

s = theis_drawdown(Q, T, r, S, t)
ax = plt.subplot(1, 1, 1, aspect="equal")
ax.contour(xg, yg, s)
ax.set_title("Theis Drawdown")

## Step 6. Create an animation of two-dimensional drawdown versus time

```
import matplotlib.animation

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect="equal")
plt.xlabel(r'x')
plt.ylabel(r'y')
times = np.arange(1, 100, 1)
title = ax.set_title(f"Time = {0}")
levels = np.linspace(.1, 1, 10)
cont = ax.contourf(xg, yg, xg * 0., levels=levels)

# animation function
def animate(i):
    global cont
    for c in cont.collections:
        c.remove()
    t = times[i]
    ax.set_title(f"Time = {t} days")
    s = theis_drawdown(Q, T, r, S, t)
    cont = ax.contourf(xg, yg, s, levels=levels)
    return cont  

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=times.shape[0])
plt.close()

from IPython.display import HTML
HTML(anim.to_jshtml())
```

In [ ]:
import matplotlib.animation
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect="equal")
plt.xlabel(r'x')
plt.ylabel(r'y')
times = np.arange(1, 100, 1)
title = ax.set_title(f"Time = {0}")
levels = np.linspace(.1, 1, 10)
cont = ax.contourf(xg, yg, xg * 0., levels=levels)

# animation function
def animate(i):
    global cont
    for c in cont.collections:
        c.remove()
    t = times[i]
    ax.set_title(f"Time = {t} days")
    s = theis_drawdown(Q, T, r, S, t)
    cont = ax.contourf(xg, yg, s, levels=levels)
    return cont  

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=times.shape[0])
plt.close()

from IPython.display import HTML
HTML(anim.to_jshtml())

## Step 7.  Create a drawdown map for multiple wells

Using the wells in the following table, create a two-dimensional drawdown map

| Name | x | y | Pumping Rate|
| --- | --- | --- | --- |
| WELL1 | 49988.2 | 40903.66 | 605.0 |
| WELL2 | 42195.49 | 5996.99 | 12.0 |
| WELL3 | 14583.68 | 15884.9 | 716.0 |
| WELL4 | 34448.56 | 27964.24 | 334.0 |
| WELL5 | 22419.85 | 31224.71 | 100.0 |
| WELL6 | 32417.15 | 4822.61 | 439.0 |
| WELL7 | 16320.24 | 13385.98 | 694.0 |
| WELL8 | 45323.84 | 36436.13 | 651.0 |
| WELL9 | 28248.69 | 39668.15 | 558.0 |
| WELL10 | 11045.92 | 31436.03 | 418.0 |
| WELL11 | 10566.4 | 8672.29 | 730.0 |
| WELL12 | 695.16 | 33597.84 | 268.0 |
| WELL13 | 9036.03 | 2583.99 | 312.0 |
| WELL14 | 44124.26 | 35123.48 | 483.0 |
| WELL15 | 22434.9 | 35106.7 | 845.0 |
| WELL16 | 22566.33 | 33533.98 | 506.0 |
| WELL17 | 2285.95 | 1383.14 | 62.0 |

In addition to the values in the table, using the following parameters

```
T = 1000. # feet squared per day
S = 0.0001 # unitless
t = 10. # days
```

In [ ]:
well_text = """WELL1 49988.2 40903.66 605.0
WELL2 42195.49 5996.99 12.0
WELL3 14583.68 15884.9 716.0
WELL4 34448.56 27964.24 334.0
WELL5 22419.85 31224.71 100.0
WELL6 32417.15 4822.61 439.0
WELL7 16320.24 13385.98 694.0
WELL8 45323.84 36436.13 651.0
WELL9 28248.69 39668.15 558.0
WELL10 11045.92 31436.03 418.0
WELL11 10566.4 8672.29 730.0
WELL12 695.16 33597.84 268.0
WELL13 9036.03 2583.99 312.0
WELL14 44124.26 35123.48 483.0
WELL15 22434.9 35106.7 845.0
WELL16 22566.33 33533.98 506.0
WELL17 2285.95 1383.14 62.0
"""

# Create the list of wells
well_list = []
for w in well_text.splitlines():
    w = w.split()
    t = (w[0], float(w[1]), float(w[2]), float(w[3]))
    well_list.append(t)
well_list

In [ ]:
def get_drawdown(wells, t, xg, yg):
    drawdown = xg * 0.
    for well in wells:
        name, xwell, ywell, rate = well
        r = np.sqrt((xg - xwell) ** 2 + (yg - ywell) ** 2)
        s = theis_drawdown(rate, T, r, S, t)
        drawdown += s
    return drawdown

x = np.linspace(-50000, 100000, 200)
y = np.linspace(-50000, 100000, 200)
xg, yg = np.meshgrid(x, y)

T = 1000. # feet squared per day
S = 0.0001 # unitless
t = 10. # days

s = get_drawdown(well_list, t, xg, yg)
ax = plt.subplot(1, 1, 1, aspect="equal")
ax.contour(xg, yg, s)
ax.set_title("Theis Drawdown")

In [ ]:
import matplotlib.animation

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect="equal")
plt.xlabel(r'x')
plt.ylabel(r'y')
times = np.arange(1, 100, 1)
title = ax.set_title(f"Time = {0}")
levels = np.linspace(.1, 2, 20)
cont = ax.contourf(xg, yg, xg * 0., levels=levels)

# animation function
def animate(i):
    global cont
    for c in cont.collections:
        c.remove()
    t = times[i]
    ax.set_title(f"Time = {t} days")
    s = get_drawdown(well_list, t, xg, yg)
    cont = ax.contourf(xg, yg, s, levels=levels)
    return cont  

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=times.shape[0])
plt.close()

from IPython.display import HTML
HTML(anim.to_jshtml())

# can use this command to write animation to file
#anim.save("animation.avi")